Construcción del pipeline Bonus 2

In [4]:
import pandas as pd
import nltk
from nltk import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import *
from nltk.stem.porter import *
from nltk import FreqDist
from nltk.tokenize import word_tokenize
from stop_words import get_stop_words
import re
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [13]:
def pipeline(dataset_location, n_samples, n_top):
    
    # Imput: 
    
        # datase_location: ubicación del archivo
        # n_samples: subset del dataset con el que trabajar, nº de filas.
        # n_top: límite de palabras top a incluir en el bag of words
    
    
    
    # Output:
        # output the classifier
    
    # CARGA DEL DATASET
    
    sentiment = pd.read_csv(dataset_location)
    dfsample = sentiment.sample(n=n_samples)
    
    # FUNCIONES PARA EL CLEANING DEL TEXTO

    def clean_up(s):

        s = re.sub(r'https?://(?:[-\w.]|(?:%[\da-fA-F]{2}))+', '', s)

        s = re.sub('[^a-zA-Z_]+', ' ', s)

        s = s.lower()

        return s



    def tokenize(s):
        """
        Tokenize a string.

        Args:
            s: String to be tokenized.

        Returns:
            A list of words as the result of tokenization.
        """
        return word_tokenize(s)


    def stem_and_lemmatize(l):

        result = []

        for i in l:

            sno = nltk.stem.SnowballStemmer('english')

            x = sno.stem(i)

            lemmatizer = WordNetLemmatizer()

            x = lemmatizer.lemmatize(x)

            result.append(x)

        return result

    def remove_stopwords(l):

        stop_words = get_stop_words('en')

        filtered_words = [word for word in l if word not in stop_words]

        return filtered_words
    
    # EJECUCIÓN DE FUNCIONES DE LIMPIEZA DE TEXTO
    
    dfsample['text_processed'] = dfsample['text'].apply(clean_up)
    dfsample['text_processed'] = dfsample['text_processed'].apply(tokenize)
    dfsample['text_processed'] = dfsample['text_processed'].apply(stem_and_lemmatize)
    dfsample['text_processed'] = dfsample['text_processed'].apply(remove_stopwords)
    
    words = []
    for x in dfsample.text_processed:
        words += x
    
    fdist = FreqDist(words)

    voc = fdist.most_common(n_top)

    bag_of_words = [x[0] for x in voc]
    
    def find_features(document):
        words = set(document)
        features = {}
        for w in top:
            features[w] = (w in words)
        s = SentimentIntensityAnalyzer().polarity_scores(" ".join(document))
        if s["pos"] > 0.2:
            s = True
        else:
            s = False
        return list((features, s))
    
    # CREACIÓN DEL BAG OF WORDS
    
    words = []
    for x in dfsample.text_processed:
        words += x

    fdist = FreqDist(words)

    voc = fdist.most_common(n_top)

    bag_of_words = [x[0] for x in voc]
    
    
    # CREACIÓN DEL DICCIONARIO PARA APLICAR nltk.NaiveBayesClassifier
    
    def find_features(document):
        words = set(document)
        features = {}
        for w in bag_of_words:
            features[w] = (w in words)
        s = SentimentIntensityAnalyzer().polarity_scores(" ".join(document))
        if s["pos"] > 0.2:
            s = True
        else:
            s = False
        return list((features, s))
    
    feature_in_text = dfsample['text_processed'].apply(find_features)
    
    # CREACIÓN DE TRAINING Y TEST SET
    
    training_set = feature_in_text[:int(n_samples*0.8)]
    testing_set = feature_in_text[int(n_samples*0.8):]
    
    # GENERACIÓN DEL CLASSIFIER
    
    classifier = nltk.NaiveBayesClassifier.train(training_set)
    
    return ("Classifier accuracy percent:",(nltk.classify.accuracy(classifier, testing_set))*100)
    

Optimización del modelo Bonus 1


In [14]:
print(pipeline('./Sentiment140.csv/Sentiment140.csv', 1000, 7500))

('Classifier accuracy percent:', 75.5)
